# Quantum Circuit Simulator Implementation

## Description :
    We have to define a circuit that consists of a list of gates with a valid format, total number of qubits then:
    1. The ground state will be calculated according to the number of qubits.
    2. The final state will be calculated .
    3. Use a weighted random technique (i.e. choices)to get counts of each state in the superposition in a certain number of shots.
    4. Define parameters for the parametric gate u3 to be included in the circuit.
    5. Define the states that we want their counts be minimum
    6. We use scipy.optimize.minimize with method "powell" to get a better value for parameters that makes the counts of unwanted states minimum
    

In [1]:
import numpy as np
from math import sin, cos, pi
from cmath import exp
from inspect import isfunction
from random import choices
from scipy.optimize import minimize

## Construct the basis gates in our simulator
    Our simulator contains identity, X, Hadmard, u3(theta, phi, lambda) gates. 

In [2]:
def identity() :
    return np.identity(2)

def x() :
    return np.array([
        [0, 1],
        [1, 0]
    ])

def h() :
    return np.array([
        [1/np.sqrt(2), 1/np.sqrt(2)],
        [1/np.sqrt(2), -1/np.sqrt(2)]
    ])

def u3(params) :
    return np.array([
        [round(cos(params["theta"] / 2.0), 4), -1 * exp(1j * params["lambda"]) * round(sin(params["theta"] / 2.0), 4)],
        [exp(1j * params["phi"]) * round(sin(params["theta"] / 2.0), 4), exp(1j * params["lambda"] + 1j * params["phi"]) * round(cos(params["theta"] / 2.0), 4)]
    ])

## Define circuit validation functions

In [3]:
def is_valid_params(keys, params) :
    is_valid = True
    for key in keys :
        if not ( key in params and isinstance(params[key], (int, float))) :
            is_valid = False
    return is_valid

def is_valid_controls(controls, total_qubits) :
    is_valid = True
    if not isinstance(controls, list) :
        is_valid = False
    else :
       for control in controls :
           if not (isinstance(control, int) and (control < total_qubits) and (control >= 0)) :
               is_valid = False
    return is_valid 
   
def is_valid_circuit(circuit, total_qubits) :
    is_valid = True
    for gate in circuit :
        if not ( "gate" in gate and "target" in gate and isfunction(gate["gate"]) and 
                isinstance(gate["target"], int) and (gate["target"] < total_qubits) and (gate["target"] >= 0)) :
            is_valid = False
        elif gate["gate"] == u3 :
            if("params" in gate) :
                is_valid = is_valid_params(np.array(["theta", "phi", "lambda"]), gate["params"])
            else :
                is_valid = False
        elif("controls"  in gate) :
            is_valid = is_valid_controls(gate["controls"], total_qubits)
    return is_valid 

## Helper functions

In [4]:
# Get projection operator |0><0|
def p0x0() :
    return np.array([
        [1, 0],
        [0, 0]
    ])

# Get projection operator |1><1|
def p1x1() :
    return np.array([
        [0, 0],
        [0, 1]
    ])
    
def get_ground_state(num_qubits) :
    # return vector of size 2**num_qubits with all zeroes except first element which is 1
    psi = np.empty(2 ** num_qubits)
    psi.fill(0)
    psi[0] = 1
    return psi

def calculate_tensor(operators) :
    # the operators are arranged so we can calculate the tensor product as follows:
    tensor = np.kron(operators[0], operators[1])
    remaining_operators = np.delete(operators, [0, 1], 0)
    for operator in remaining_operators :
        tensor = np.kron(tensor, operator)
    return tensor 

def get_opertors_within_controlled_gate(controlled_gate_details) :
    first_control_pos = controlled_gate_details["first_control_pos"]
    total_qubits = controlled_gate_details["total_qubits"]
    operators = {0: np.empty(total_qubits, object), 1: []}
    # in operators[0] array we append |0><0| in the position of first control (first control is first control from
    # upper qubit in case the control position < target position otherwise the first control is the first control
    # from the bottom qubit) and append I in all other positons; that's to represent the case if the first control 
    # is |0> then we apply nothing on target and all other qubits
    operators[0].fill(identity())
    operators[0][first_control_pos] = p0x0()
    # in operators[1] array we append |1><1| in the position of first control and append unitary gate in the position 
    # of target,but, if the gate is multi-controlled, we recursively calculate the operator of the n-1 controlled gate 
    # and append the operator in the position of the next control; that's to represent the case if the first control 
    # is |1> then we apply the gate on target and I on other qubits that are not control nor target
    for qubit_index in controlled_gate_details["qubit_range"] :
        if qubit_index == first_control_pos :
            operators[1].append(p1x1())
        else : # qubit doesn't have control nor target
            operators[1].append(identity())
    remaining_controls = controlled_gate_details["controls"].copy()
    remaining_controls.remove(first_control_pos) 
    if controlled_gate_details["is_reversed_gate"] == False :
        # update positions of the controls to calculate the operator of the n-1 controlled gate separately from the 
        # circuit
        remaining_controls[:] = [
            remaining_control - (first_control_pos + 1) for remaining_control in remaining_controls
        ]
    # here we append the operator in the position of the next control, for example : append cnot if the gate was toffoli
    operators[1].append(get_operator(
        controlled_gate_details["next_total_qubits"], 
        controlled_gate_details["gate_unitary"], 
        controlled_gate_details["target_qubit"], remaining_controls
    ))
    if controlled_gate_details["is_reversed_gate"] == True :
        operators[1].reverse()
    return operators
    
def get_operator(total_qubits, gate_unitary, target_qubit, controls = []):
    # return unitary operator of size 2**n x 2**n for given gate and controls and target qubits
    # We handle cases that the gate is single qubit , controlled, multi-controlled with the target is at any qubit
    if controls :
        controls.sort()
        if controls[0] < target_qubit :
            #normal flow
            first_control_pos = controls[0]
            controlled_gate_details = {
                "controls": controls,
                "first_control_pos": first_control_pos,
                "total_qubits": total_qubits,
                "next_total_qubits": total_qubits - (first_control_pos + 1),
                "gate_unitary": gate_unitary,
                "target_qubit": target_qubit - (first_control_pos + 1),
                "is_reversed_gate": False,
                "qubit_range": range(first_control_pos + 1),
                "pos_after_control": first_control_pos + 1
            }             
        else :
            #the controlled gate is upside down
            controls.sort(reverse = True)
            last_control_pos = controls[0]
            controlled_gate_details = {
                "controls": controls,
                "first_control_pos": last_control_pos,
                "total_qubits": total_qubits,
                "next_total_qubits": last_control_pos,
                "gate_unitary": gate_unitary,
                "target_qubit": target_qubit,
                "is_reversed_gate": True,
                "qubit_range": range(total_qubits - 1, last_control_pos - 1, -1),
                "pos_after_control": last_control_pos - 1
            }
        operators = get_opertors_within_controlled_gate(controlled_gate_details)
        operator = calculate_tensor(operators[0]) + calculate_tensor(operators[1])
                  
    else :   
        operators = np.empty(total_qubits, object)
        operators.fill(identity())
        operators[target_qubit] = gate_unitary 
        if total_qubits == 1 :
            operator = gate_unitary
        else :
            operator = calculate_tensor(operators)
        
    return operator

def get_gate_unitary(gate) :
    params = gate.get("params")
    if params : # parametric gate (i.e. u3)
        gate_unitary = gate["gate"](params)
    else : 
        gate_unitary = gate["gate"]()
    return gate_unitary

def run_program(program) :
    # read program, and for each gate:
    #   - calculate matrix operator
    #   - multiply state with operator
    # return final state
    state = program["initial_state"]
    for gate in program["circuit"] :
        gate_unitary = get_gate_unitary(gate)
        controls = gate.get("controls") or []
        operator = get_operator(program["total_qubits"], gate_unitary, gate["target"], controls)
        state = np.dot(state, operator)
    return state

def measure_all(binary_states, probabilities, num_shots)  :
    # get num_shots random results weighted by probabilities
    random_results = choices(binary_states, probabilities, k=num_shots) 
    statistics = {}
    for binary_state in binary_states :
        statistics[binary_state] = random_results.count(binary_state)
    return statistics

def get_counts(total_qubits, state_vector, num_shots) :
    # return object with statistics in following form:
    #   {
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      ...
    #   }
    # (only for elements which occoured - returned from measure_all)
    binary_states = np.array([])
    probabilities = np.array([])
    for qubit_index in range(2 ** total_qubits) :
        if(state_vector[qubit_index] != 0) :
            binary_states = np.append(binary_states, format(qubit_index, "0" + str(total_qubits) +"b"))
            probabilities = np.append(probabilities, np.abs(state_vector[qubit_index]) ** 2 )  
    return measure_all(binary_states, probabilities, num_shots) 


## Get Circuit depending on parameters
- It should be a list of dictionaries, each with key "gate" that must be defined in our basis gates above, and key "target" that is the index of the target qubit (target means the qubit on which the single qubit gate is applied). 
- If the gate is controlled, then you can add a "controls" key which is a list of control indices that can't be equal to target qubit index.
- In the u3 parametric gate, the "params" key should be defined as a dictionary of params which are "lambda", "theta" and "phi"


In [5]:
def circuit(parameters = []) : # parameters is optional if we don't want to use parametric gate
    return [
    #{ "gate": x, "target": 1, "controls": [0,2]}, # means CNOT gate with control on qubit 0 and 2 and target on qubit 1, where total qubits > 2
    #{ "gate": h, "target": 0}, # Apply hadmard gate on qubit 0
    { "gate": u3, "params": { "theta": parameters[0], "phi": parameters[1], "lambda": 0 }, "target": 0}, # Apply u3(theta, phi, lambda) gate on qubit 0
    { "gate": x, "target": 1, "controls": [0]} # means CNOT gate with control on qubit 0 and target on qubit 1
    ]

## Define cost function 

In [6]:
def calculate_cost(counts, states_to_minimize) :
    # the cost is the sum of counts of unwanted states
    cost = 0
    for state in states_to_minimize :
        if state in counts :
            cost = cost + counts[state] 
    return cost

## Define objective function

In [7]:
def objective_function(params, counts, states_to_minimize) :
    # calculate cost & return it
    return calculate_cost(counts, states_to_minimize)

## Define program:

1. Total qubits :

In [8]:
total_qubits = 2

2. The initial values of u3 parameters which are the angle values theta and phi to be passed to the circuit function


In [9]:
# initial values
parameters = np.array([10.99433535,  7.8527427])

3. Define states that we want their counts be nearly zero
(here we want states 01 and 10 and write them in binary format)

In [10]:
states_to_minimize = np.array([format(1, "02b"), format(2, "02b")])

4. Define the circuit (you should pass parameters if you want to use u3 gate)

In [11]:
my_circuit = circuit(parameters)

5. If the circuit is valid, the program should calculate ground and final states and use a weighted random technique to get counts of each state in the superposition in a certain number of shots, else error msg will be returned
**Note that We are dealing with "big endian" encoding**

In [12]:
if (is_valid_circuit(my_circuit, total_qubits)) :
    my_qpu = get_ground_state(total_qubits)
      
    # Run circuit
    program = {"initial_state": my_qpu, "circuit": my_circuit, "total_qubits": total_qubits}
    final_state = run_program(program)
    
    # Read results
    counts = get_counts(total_qubits, final_state, 1000)
    print("counts:", counts, "\n")
    
    # minimize (trying to minimize states that we don't want)
    minimum = minimize(objective_function, parameters, args = (counts, states_to_minimize), method = "Powell", tol = 1e-6)
    print ("minimization results:\n", minimum)
else :
    print("Please try again with a valid circuit format")

counts: {'00': 526, '11': 474} 

minimization results:
    direc: array([[1., 0.],
       [0., 1.]])
     fun: array(0)
 message: 'Optimization terminated successfully.'
    nfev: 43
     nit: 1
  status: 0
 success: True
       x: array([13.61191625, 10.4703236 ])
